In [1]:
import os
import numpy as np
import pandas
import os
import skimage
from scipy import ndimage, signal
from skimage import exposure
import matplotlib.pyplot as plt
import glob
from skimage import io
import scipy
from scipy import signal
from skimage import feature
from tqdm import tqdm_notebook as tqdm
import json
from matplotlib_scalebar.scalebar import ScaleBar
import networkx as nx

In [2]:
def default(o):
    if isinstance(o, np.int64): return int(o)
    raise TypeError

In [9]:
def _makeSpheroidClass(path, zRatio, rNoyau, dCells):
    
    for spheroidFolder in tqdm(os.listdir(path)):
        
        spheroidPath = path + '/' + spheroidFolder
        
        if os.path.isdir(spheroidPath):
            
            for timeFolder in os.listdir(spheroidPath):
            
                timePath = spheroidPath + r'\\' + timeFolder
                
                if os.path.isdir(timePath):
                                                
                    print('prep image: ' + spheroidFolder + ' folder and time ' + timeFolder)

                    Sph = spheroid(timePath, spheroidFolder, timeFolder, zRatio, rNoyau, dCells)
                    
                    try:

                        Sph._loadImageNuclei(1)
                        Sph._loadImageDead(0)

                        print('image made, starting nuclei ID')

                        Sph._getNuclei()

                        print('nuclei gotten, make spheroid')

                        Sph._makeSpheroid()

                        Sph._initializeDead()

                        with open(path + '\spheroid_' + spheroidFolder + r'_' +  timeFolder + '.json', 'w') as fp:

                            json.dump(Sph.Spheroid, fp, default = default)

                        #Sph._verifySegmentation()
                        
                    except: print('Error on: '+ spheroidFolder + ' folder and time ' + timeFolder)

In [10]:
def _sortFiles(path):
    
    for fileName in tqdm(os.listdir(path)):
        
        if not os.path.isdir(path + r'\\' + fileName):
            
            _, position = fileName.split('xy')
            position, time, _ = position.split('t')
            time, _ = time.split('z')

            if not os.path.exists(path + r'\\' + position):
                os.mkdir(path + r'\\' + position)

            if not os.path.exists(path + r'\\' + position + r'\\' + time):
                os.mkdir(path + r'\\' + position + r'\\' + time)

            os.rename(path + r'\\' + fileName, path + r'\\' + position + r'\\' + time + r'\\' + fileName)

    return print('job done')

In [11]:
class spheroid:
    
    """ Spheroid class containing the necessary info to build the spheroid.
    
    ====== NOTICE ======
    
     - All variables starting with a capital letter refer to class variables.
     - All variables starting with '_' refer to a dict
    
    
    ====== PARAMETERS ======
    
    path: string object, path to the folder storing images
    position: string object, well ID
    time: string object, time of experiment"""
    
    def __init__(self, path, position, time, zRatio, rNoyau, dCells):
        
        self.Path = path
        self.Position = position
        self.Time = time
        self.ZRatio = zRatio
        self.RNoyau = rNoyau
        self.DCells = dCells
        self.NucImage = []
        self.DeadImage = []
        self.Thresh = 500
        self.ThreshCell = 200
        
    def _loadSpheroid(self, spheroidDict):
        
        self.spheroid = spheroidDict
        
    
    def _loadImageNuclei(self, channelNuc):
        
        image_list = []    
        for filename in sorted(os.listdir(self.Path)): #assuming tif
            
            if '.tif' in filename:
                        
                im = io.imread(self.Path + '/' + filename)
                image_list.append(im[channelNuc])
        
        self.NucImage = np.reshape(image_list, (len(image_list), np.shape(image_list[0])[0], np.shape(image_list[0])[1]))

    def _loadImageDead(self, channelDead):
        
        image_list = []    
        for filename in sorted(os.listdir(self.Path)): #assuming tif
            
            if '.tif' in filename:
                        
                im = io.imread(self.Path + '/' + filename)
                image_list.append(im[channelDead])
        
        self.DeadImage = np.reshape(image_list, (len(image_list), np.shape(image_list[0])[0], np.shape(image_list[0])[1]))
    
    def _getNuclei(self):
        
        if not len(self.NucImage):
            
            ### Check that image to study does exist
            
            print('Image doesnt exist')
        
        self._getMaximaFrame(self._getMaskImage(), self.ZRatio, self.RNoyau)
        self._duplicataClean()
        
    
    def _makeSpheroid(self):
        
        ### COPYPASTE ABOVE
        
        if not len(self.NucFrame):
            
            ### Check that image to study does exist
            
            print('Image doesnt exist')
                
        """Generates the spheroid dict containing all the essential information about the spheroid.

        ====== PARAMETERS ======

        df: DataFrame containing all the positional information of the cells
        dCells: maximum distance for two cells to be considered as neighbours
        zRatio: pixel ratio between z and xy dimensions
        Image: original, multichannel, 3D image
        state: True/False variable stating if we seek the dead-alive information

        ====== RETURNS ======

        _Spheroid: dict object"""
    
        _Spheroid = {}
    
        _Spheroid['spheroid position'] = self.Position
        _Spheroid['time'] = self.Time
        _Spheroid['cells'] = self._generateCells()
            
        self.Spheroid = _Spheroid
        
        
    def _initializeDead(self):
            
        X = np.arange(0, 40)
        Y = np.arange(0, 40)
        Z = np.arange(0, 40)
        X, Y, Z = np.meshgrid(X, Y, Z)

        mask = np.sqrt((X-20)**2 + (Y-20)**2 + (Z-20)**2/self.ZRatio**2) < self.RNoyau
        mask = np.transpose(mask, (2,1,0)).astype(np.int)
        
        deadConv = scipy.signal.fftconvolve(self.DeadImage, mask, mode='same')

        
        for cellLabel in self.Spheroid['cells'].keys():
            
            x = self.Spheroid['cells'][cellLabel]['x']
            y = self.Spheroid['cells'][cellLabel]['y']
            z = self.Spheroid['cells'][cellLabel]['z']
            
            zlen, _, _ = np.nonzero(mask)
            
            # Test dimension order to verify coherence
            if deadConv[z,x,y]/len(zlen) > self.Thresh:
                
                print('Dead cell')
                
                self.Spheroid['cells'][cellLabel]['state'] = 'Dead'
    
    def _verifySegmentation(self):
        
        if not len(self.NucFrame):
            return print('Image doesnt exist')
        
        if not os.path.exists(self.Path + r'/filmstack/'):
            os.mkdir(self.Path + r'/filmstack/')
        
        zshape, _, _ = np.shape(self.NucImage)
        
        ImageAll = self.NucImage
                
        for n in range(zshape):
                        
            Image = ImageAll[n,:,:]
            
            plt.figure(figsize=(6, 6))
            plt.subplot(111)
            plt.imshow(Image, vmin=0, vmax=800, cmap=plt.cm.gray)
            plt.axis('off')
            
            scalebar = ScaleBar(0.0000006, location = 'lower right') # 1 pixel = 0.6 umeter
            plt.gca().add_artist(scalebar)
            
            r = self.RNoyau
            
            for item, row in self.NucFrame.iterrows():
                
                if (r**2 - (row['z'] -n)**2/self.ZRatio**2) > 0:
                    
                    rloc = np.sqrt(r**2 - (row['z'] -n)**2/self.ZRatio**2)
                    s = np.linspace(0, 2*np.pi, 100)
                    x = rloc*np.sin(s) + row['x']
                    y = rloc*np.cos(s) + row['y']

                    plt.plot(y, x, 'r-')
            
            plt.savefig(self.Path + r'/filmstack/im_' + str(n) +'.png')
            plt.close()
        
    #### UTILITY FUNCTIONS ####
    
    # Question: how store utility functions in Python class?
    
    def _getMaskImage(self):

        blurred = ndimage.gaussian_filter(self.NucImage, sigma=2)
        mask = (blurred > np.percentile(blurred, 93)).astype(np.float)
        mask += 0.1

        binary_img = mask > 0.5
        binary_img = skimage.morphology.binary_closing(ndimage.binary_dilation(ndimage.binary_erosion(binary_img)).astype(np.int))

        return np.multiply(blurred, binary_img)
    
    def _getMaximaFrame(self, Image, zRatio, rNoyau):

        """Generates the spheroid dict containing all the essential information about the spheroid.

        ====== PARAMETERS ======

        Image: original, multichannel, 3D image
        zRatio: pixel ratio between z and xy dimensions
        rNoyau: radius of the prior (i.e. expected radius of the nuclei)

        ====== RETURNS ======

        _Spheroid: dict object"""

        z, x, y = np.nonzero(Image)
        binary_img = Image > 0 # Regarder si le seuil est le bon ou pas

        mask_image_crop = Image[min(z):max(z), min(x):max(x), min(y):max(y)]

        a = mask_image_crop
        zDim, xDim, yDim = np.shape(mask_image_crop)

        X = np.arange(0, 20)
        Y = np.arange(0, 20)
        Z = np.arange(0, 20)
        X, Y, Z = np.meshgrid(X, Y, Z)

        mask = np.sqrt((X-10)**2 + (Y-10)**2 + (Z-10)**2/zRatio**2) < rNoyau
        mask = np.transpose(mask, (2,1,0))
        zlen, _, _ = np.nonzero(mask)

        conv = scipy.signal.fftconvolve(a, mask, mode='same')

        mask_conv = ndimage.gaussian_filter(np.multiply(conv, binary_img[min(z):max(z), min(x):max(x), min(y):max(y)]), 
                                            sigma=2)/len(zlen)
        
        # Distance minimum entre deux noyaux repérés par l'algorithme de convolution
        # On prend exprès 10% de marge supplémentaire pour éviter les contacts inopportuns.
        # On fait aussi attention au liveau minimal d'intensite des pics.
        
        r = int(self.RNoyau/2)
        
        coordinates = np.asarray(skimage.feature.peak_local_max(mask_conv, threshold_abs=self.ThreshCell, min_distance= r))

        coordinates[:, 0] += min(z)
        coordinates[:, 1] += min(x)
        coordinates[:, 2] += min(y)

        df = pandas.DataFrame(coordinates, columns = ['z', 'x', 'y'])

        for ind, row in df.iterrows():
            df.loc[ind, 'val'] = mask_conv[int(row['z']) - min(z), int(row['x']) - min(x), int(row['y']) - min(y)]
            df.loc[ind, 'label'] = int(ind)

        self.NucFrame = df
        
    
    def _duplicataClean(self):
        
        df = self.NucFrame

        for ind, row in df.iterrows():

            lf = df.loc[(df['x'] - row['x'])**2 + (df['y'] - row['y'])**2 < 4] # on élimine les duplicatats selon z

            if len(lf) > 1:

                a = len(df)
                df = df.drop(lf.loc[lf['val'] < lf['val'].max()].index)

        self.NucFrame = df
        
    def _generateCells(self):

        """ This function serves to generate the cells and gives back a dic object.

        ====== PARAMETERS ======

        df: DataFrame containing the relevant positional information
        dCells: minimum distance between any two cells
        zRatio: ratio between the xy and z dimensions

        ====== RETURNS ======

        _Cells: dict object"""

        _Cells = {}
        
        df = self.NucFrame
        dCells = self.DCells
        zRatio = self.ZRatio

        df['label'] = df['label'].astype(int).astype(str)

        for label in df['label'].unique():

            dic = {}

            dic['x'] = df.loc[df['label'] == label, 'x'].iloc[0]
            dic['y'] = df.loc[df['label'] == label, 'y'].iloc[0]
            dic['z'] = df.loc[df['label'] == label, 'z'].iloc[0]
            dic['neighbours'] = self._nearestNeighbour(df, label, dCells, zRatio)
            dic['state'] = 'Live'

            _Cells[str(int(label))] = dic

        return _Cells

    def _nearestNeighbour(self, df, label, dCells, zRatio):
    
        """Returns a list of float labels of the cells closest to the given label.
        This method is dependant only on a minimum distance given by the investigator."""

        x = df.loc[df['label'] == label, 'x'].iloc[0]
        y = df.loc[df['label'] == label, 'y'].iloc[0]
        z = df.loc[df['label'] == label, 'z'].iloc[0]

        lf = df.loc[df['label'] != label].copy() 

        return lf.loc[np.sqrt((lf['x'] - x)**2 + (lf['y'] - y)**2 + (lf['z'] - z)**2/zRatio**2) < dCells, 'label'].values.tolist()


In [12]:
### PAY ATTENTION TO THE CHANNEL ORDER

path = r'D:\Gustave\27052019_TL'

_makeSpheroidClass(path, 1/4, 14, 60)

prep image: 01 folder and time 1
image made, starting nuclei ID


C:\Users\gronteix\AppData\Local\Continuum\anaconda3\lib\site-packages\mkl_fft\_numpy_fft.py:1044: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfftn_numpy(a, s, axes)


nuclei gotten, make spheroid
Dead cell
Dead cell
Dead cell
Dead cell
prep image: 01 folder and time 2
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
Dead cell
Dead cell
Dead cell
prep image: 01 folder and time 3
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
Dead cell
Dead cell
Dead cell
prep image: 01 folder and time 4
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
Dead cell
Dead cell
Dead cell
prep image: 01 folder and time 5
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
Dead cell
Dead cell
Dead cell
Dead cell
prep image: 01 folder and time 6
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
Dead cell
Dead cell
Dead cell
Dead cell
Dead cell
prep image: 02 folder and time 1
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 02 folder and time 2
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 02 folder and time

nuclei gotten, make spheroid
Dead cell
prep image: 13 folder and time 4
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 13 folder and time 5
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 13 folder and time 6
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 14 folder and time 1
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 14 folder and time 2
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 14 folder and time 3
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 14 folder and time 4
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 14 folder and time 5
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 14 folder and time 6
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 15 folder and time 1
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 15 folder

nuclei gotten, make spheroid
prep image: 28 folder and time 2
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 28 folder and time 3
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 28 folder and time 4
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 28 folder and time 5
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 28 folder and time 6
Error on: 28 folder and time 6
prep image: 29 folder and time 1
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 29 folder and time 2
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 29 folder and time 3
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 29 folder and time 4
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 29 folder and time 5
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
Dead cell
prep image: 29 folder and time

image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 42 folder and time 6
Error on: 42 folder and time 6
prep image: 43 folder and time 1
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
Dead cell
prep image: 43 folder and time 2
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
Dead cell
prep image: 43 folder and time 3
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
Dead cell
prep image: 43 folder and time 4
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
Dead cell
Dead cell
prep image: 43 folder and time 5
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
Dead cell
Dead cell
prep image: 43 folder and time 6
Error on: 43 folder and time 6
prep image: 44 folder and time 1
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 44 folder and time 2
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 44 folder and time 3
image ma

image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 57 folder and time 5
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 57 folder and time 6
Error on: 57 folder and time 6
prep image: 58 folder and time 1
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 58 folder and time 2
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
Dead cell
prep image: 58 folder and time 3
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
Dead cell
prep image: 58 folder and time 4
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 58 folder and time 5
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 58 folder and time 6
Error on: 58 folder and time 6
prep image: 59 folder and time 1
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 59 folder and time 2
image made, starting nuclei ID
nuclei gotten, ma

prep image: 71 folder and time 2
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 71 folder and time 3
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 71 folder and time 4
image made, starting nuclei ID
nuclei gotten, make spheroid
prep image: 71 folder and time 5
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 71 folder and time 6
Error on: 71 folder and time 6
prep image: 72 folder and time 1
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 72 folder and time 2
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 72 folder and time 3
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 72 folder and time 4
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 72 folder and time 5
image made, starting nuclei ID
nuclei gotten, make spheroid
Dead cell
prep image: 72 folder and time 6
Error 

In [ ]:
def _cleanCells(path):
    
    for dicName in tqdm(sorted(glob.glob(path + r'/' + '*.json'))):
            
    with open(dicName, "r") as read_file:
        data = json.load(read_file)
        
    _Cells = data['cells']
    
    for cell in _Cells.keys():
        
        zList

In [ ]:
def _getLen(dic):
    
    return len(dic['cells'].keys())

In [ ]:
def _getProperties(dic):
    
    G=nx.Graph()
    
    G.add_nodes_from(dic.keys())
        
    _Cells = dic['cells']
    
    for key in _Cells.keys():
    
        neighbours = _Cells[key]['neighbours']
    
        for node in neighbours:
    
            G.add_edge(key, node)
        
    degree_sequence = sorted([d for n, d in G.degree()], reverse=True)  # degree sequence
    

    return nx.density(G), degree_sequence

In [ ]:
def _getAllGraphs(path):
    
    df = pandas.DataFrame()
    i = 0
    
    degree = []
    
    for dicName in glob.glob(path + r'/' + '*.json'):
                
        with open(dicName, "r") as read_file:
            data = json.load(read_file)
            
        _, sphNum = dicName.split('spheroid_')
        sphNum, _ = sphNum.split('.')
        
        density, deg = _getProperties(data)
        
        degree += deg
        
        df.loc[i, 'label'] = sphNum
        df.loc[i, 'density'] = density
        i += 1
        
    # print "Degree sequence", degree_sequence
    degreeCount = collections.Counter(degree)
    deg, cnt = zip(*degreeCount.items())
        
    return df, deg, cnt

In [ ]:
import collections
path1 = r'/Users/gustaveronteix/Desktop/Image Stack/Test/Screens/SIR_1mM/'
path2 = r'/Users/gustaveronteix/Desktop/Image Stack/Test/Screens/SIR_05mM/'



df1, deg1, cnt1 = _getAllGraphs(path1)
df2, deg2, cnt2 = _getAllGraphs(path2)

In [ ]:
fig, ax = plt.subplots()
plt.bar(deg, cnt, width=0.80, color='b')

plt.title("Degree Histogram")
plt.ylabel("Count")
plt.xlabel("Degree")
ax.set_xticks([d + 0.4 for d in deg])
ax.set_xticklabels(deg)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import math

# poisson function, parameter lamb is the fit parameter
def poisson(k, lamb):
    return (lamb**k/math.factorial(k))*np.exp(-lamb)

l = np.dot(np.asarray(list(deg)),np.asarray(list(cnt)))/np.sum(np.asarray(list(cnt)))

plt.plot(deg, poisson(np.asarray(list(deg)), l), 'r-', lw=2)
plt.show()

In [ ]:
%matplotlib
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# What follows is a copy of the 3D plot example code.
# Data is randomly generated so there is no external data import.

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(df['x'], df['y'], df['z'])

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_zlim(0,51)

plt.show()


from matplotlib_scalebar.scalebar import ScaleBar

for n in range(len(mask_img)):

    plt.subplot(111)
    plt.imshow(mask_img[n], cmap=plt.cm.gray, alpha = 0.8)
    plt.axis('off')
    
    scalebar = ScaleBar(0.000001, location = 'lower right') # 1 pixel = 0.2 meter
    plt.gca().add_artist(scalebar)

    lf = df.loc[df['z'] == n]
    mf = df.loc[df['z'] == n-1]
    pf = df.loc[df['z'] == n+1]

    plt.plot(mf['y'], mf['x'], 'go', label = 'z = ' + str(n-1))
    plt.plot(lf['y'], lf['x'], 'yo', label = 'z = ' + str(n))
    plt.plot(pf['y'], pf['x'], 'ro', label = 'z = ' + str(n+1))

    plt.show()
    plt.legend()
    
    plt.savefig(r'/Users/gustaveronteix/Desktop/Image Stack/filmstack/im_' + str(n) +'.png')
    plt.close()

In [ ]:
import networkx as nx
G=nx.Graph()

In [ ]:
G.add_nodes_from(_Cells.keys())

In [ ]:
for key in _Cells.keys():
    
    neighbours = _Cells[key]['neighbours']
    
    for node in neighbours:
    
        G.add_edge(key, node)

In [ ]:
zAlt = []

for key in _Cells.keys():
        
    zAlt.append(_Cells[key]['z'])

In [ ]:
plt.figure()

pos = nx.spring_layout(G)
im = nx.draw_networkx_nodes(G, pos, node_size=20, node_color = zAlt, cmap=plt.cm.viridis)
nx.draw_networkx_edges(G, pos, alpha=0.4)
plt.colorbar(im)
plt.show()